In [30]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

## Parametri

In [31]:
folder = os.path.join("..","processed_data")
savefolder = 'windowed_data_se'
tw = [2015,2022]
windowsize = 3

## Caricamento e utils

In [32]:
def temporalmerge(df1, temporaldf, columns):
    for prof in tqdm( df1.ID.unique() ):
        df2 = df1
        idx = df1.loc[df1.ID == prof].index
        temp = temporaldf.loc[temporaldf.ID == prof]

        for column in columns:
            columndict = {}

            for index,row in temp.iterrows():
                #each row is a year
                try:
                    columndict[row['anno']] = row[column]
                except:
                    columndict[row['year']] = row[column]

            for year in sorted( columndict.keys() ):
                df2.loc[idx, column +  "_" + str(year)] = columndict[year]

    df2.fillna(0, inplace=True)

    for col in df2.columns:
        try:
            if 'mean' not in col:
                df2[col] = df2[col].astype(int)
        except:
            continue

    return df2

In [33]:
settoriscientifici = pd.read_csv(os.path.join("..","data","settori_scientifici.csv"), index_col = 0).reset_index(drop=True)

In [34]:
docenti = pd.read_csv(os.path.join(folder,'docenti.csv'), index_col = 0)
afferisce = pd.read_csv(os.path.join(folder,'afferisce.csv'), index_col = 0)
ateneo = pd.read_csv(os.path.join(folder,'ateneo.csv'), index_col = 0)
fascia = pd.read_csv(os.path.join(folder,'fascia.csv'), index_col = 0)
lavora = pd.read_csv(os.path.join(folder,'lavora.csv'), index_col = 0)
ruolo = pd.read_csv(os.path.join(folder,'ruolo.csv'), index_col = 0)
scopus_static = pd.read_csv(os.path.join(folder,'scopus_static.csv'), index_col = 0)
scopus_yby = pd.read_csv(os.path.join(folder,'scopus_yby.csv'), index_col = 0)
settore = pd.read_csv(os.path.join(folder,'settore.csv'), index_col = 0)
settoreconcorsuale = pd.read_csv(os.path.join(folder,'settoreconcorsuale.csv'), index_col = 0)
strutturaafferenza = pd.read_csv(os.path.join(folder,'strutturaafferenza.csv'), index_col = 0)


## Costruzione DataFrame Italy

In [35]:
import re
scholarfolder = os.path.join('..','data','scholar')
scholarfiles = os.listdir(scholarfolder)
print(scholarfiles)

['authors_sa.csv', 'authors_mde.csv', 'authors_st.csv', 'authors_sq.csv', 'authors_se.csv']


In [36]:
def split_name(name, return_first=True):
    pattern = r'(.+?)[ -]*([A-Z][a-z]+ ?[A-Z]*[a-z]*)'
    prof = name
    try:
        first_name = (re.findall(pattern, prof)[0])[0].replace(u'\xa0', u'')
        last_name = (re.findall(pattern, prof)[0])[1]
        if return_first:
            return first_name
        else:
            return last_name
    except:
        prof = prof.replace('é','e')
        prof = prof.replace('Ì', 'I')
        try:
            first_name = (re.findall(pattern, prof)[0])[0].replace(u'\xa0', u'')
            last_name = (re.findall(pattern, prof)[0])[1]
            if return_first:
                return first_name
            else:
                return last_name
        except:
            return "n"

In [37]:
##prepare software_engineers

software_engineers = pd.read_csv(os.path.join(scholarfolder, "authors_se.csv"))
software_architects = pd.read_csv(os.path.join(scholarfolder, "authors_sa.csv"))
model_driven_engineers = pd.read_csv(os.path.join(scholarfolder, "authors_mde.csv"))
software_quality = pd.read_csv(os.path.join(scholarfolder, "authors_sq.csv"))
software_testing = pd.read_csv(os.path.join(scholarfolder, "authors_st.csv"))

#make list of all
areas_list = [software_engineers,software_architects,model_driven_engineers,software_quality,software_testing]

for al in areas_list:
    for index, row in al.iterrows():

        #Remove parenthesis and content from column name
        al.loc[index,'Name'] = re.sub(r'\([^)]*\)', '', row['Name'])

        al.loc[index,'First_Name'] = split_name(row['Name'],return_first=True)
        al.loc[index,'Surname'] = split_name(row['Name'],return_first=False)

        #lowercase First_Name and Surname
        al.loc[index,'Surname'] = al.loc[index,'Surname'].lower()
        al.loc[index,'First_Name'] = al.loc[index,'First_Name'].lower()

software_engineers = software_engineers[['Name','First_Name','Surname']]
software_architects = software_architects[['Name','First_Name','Surname']]
model_driven_engineers = model_driven_engineers[['Name','First_Name','Surname']]
software_quality = software_quality[['Name','First_Name','Surname']]
software_testing = software_testing[['Name','First_Name','Surname']]

se_total = pd.concat([software_engineers,software_architects,model_driven_engineers,software_quality,software_testing],axis=0,ignore_index=True)

se_total = se_total.drop_duplicates(subset=['Name','First_Name','Surname'], keep='first')

In [38]:
docenti['First_Name'] = docenti['First_Name'].str.lower()
docenti['Surname'] = docenti['Surname'].str.lower()

In [39]:
docenti = docenti.merge(se_total, on=['First_Name','Surname'])

In [40]:
f = lambda x: str( x[1].split(' ')[0])
settoriscientifici['S.S.D.'] = settoriscientifici.apply(f, axis=1)
settoriscientifici = settoriscientifici[['Cod_AreaCUN','S.S.D.']]

In [41]:
docenti = docenti.merge(settoriscientifici)

In [42]:
lavora = lavora.drop(columns=['ateneo ID'])
lavmin = lavora.groupby(['ID'])['anno'].min().reset_index()
lavmin.rename(columns={'anno':'inizio'},inplace=True)
lavmax = lavora.groupby(['ID'])['anno'].max().reset_index()
lavmax.rename(columns={'anno':'fine'},inplace=True)
lavdur = lavora.groupby(['ID'])['anno'].count().reset_index()
lavdur.rename(columns={'anno':'durata'},inplace=True)

In [43]:
lavora = lavmin.merge(lavmax)
lavora = lavora.merge(lavdur)

In [44]:
lavora

,ID,inizio,fine,durata
0,0,2012,2022,11
1,1,2012,2022,11
2,2,2012,2022,11
3,3,2019,2022,4
4,4,2012,2022,11
...,...,...,...,...
84724,84724,2014,2016,3
84725,84725,2016,2017,2
84726,84726,2014,2015,2
84727,84727,2015,2015,1


In [45]:
lavora = lavora.loc[(lavora.inizio <= tw[0]) & (lavora.fine >= tw[1]) & (lavora.durata >= (tw[1]-tw[0]+1))]
lavora

,ID,inizio,fine,durata
0,0,2012,2022,11
1,1,2012,2022,11
2,2,2012,2022,11
4,4,2012,2022,11
5,5,2012,2022,11
...,...,...,...,...
84682,84682,2012,2022,10
84684,84684,2012,2022,11
84685,84685,2014,2022,9
84686,84686,2014,2022,9


In [46]:
docenti = docenti.merge(lavora)
#docenti

Aggiungiamo i dati di SCOPUS

In [47]:
#docentiscopus = docenti.merge(scopus_static)

#total = docenti.merge(scopus_static)

total = docenti
total = temporalmerge(total,scopus_yby, ['papers', 'citations'])


  0%|          | 0/72 [00:00<?, ?it/s]/var/folders/1f/dn5t35316d5b0gv0v6mcj6sm0000gn/T/ipykernel_4280/398045178.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df2.loc[idx, column +  "_" + str(year)] = columndict[year]
/var/folders/1f/dn5t35316d5b0gv0v6mcj6sm0000gn/T/ipykernel_4280/398045178.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df2.loc[idx, column +  "_" + str(year)] = columndict[year]
/var/folders/1f/dn5t35316d5b0gv0v6mcj6sm0000gn/T/ipykernel_4280/398045178.py:18: PerformanceWarning: DataFrame is highly fra

In [48]:
r = ruolo.merge(fascia)
total = temporalmerge(total, r, ['fascia_id'])
gendict = {'F':0, 'M':1}
total['sex'] = total['Genere']
total.replace( {"sex":gendict}, inplace=True )

100%|██████████| 72/72 [00:00<00:00, 490.67it/s]
/var/folders/1f/dn5t35316d5b0gv0v6mcj6sm0000gn/T/ipykernel_4280/3813587996.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  total['sex'] = total['Genere']


## Anonimizzazione

In [49]:
total.drop(columns = ['Cognome e Nome', 'Genere', 'Surname', 'First_Name', 'ID'], inplace=True)

In [50]:
total = total.loc[total.Cod_AreaCUN.isin([1,9])]

In [51]:
total.drop(columns=['Cod_AreaCUN'], inplace=True)

/var/folders/1f/dn5t35316d5b0gv0v6mcj6sm0000gn/T/ipykernel_4280/1059717045.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total.drop(columns=['Cod_AreaCUN'], inplace=True)


## Sliding Windows

In [52]:
total.rename(columns={'durata':'anni_servizio'}, inplace=True)
total.drop(columns={'inizio','fine'}, inplace=True)

/var/folders/1f/dn5t35316d5b0gv0v6mcj6sm0000gn/T/ipykernel_4280/3598284778.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total.rename(columns={'durata':'anni_servizio'}, inplace=True)
/var/folders/1f/dn5t35316d5b0gv0v6mcj6sm0000gn/T/ipykernel_4280/3598284778.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total.drop(columns={'inizio','fine'}, inplace=True)


In [53]:
df_yby = {}

In [54]:
for year in reversed(range( tw[0]+windowsize, tw[1]+1) ):
    yearstoconsider = list(range(year-windowsize,year+1))
    startingyear = 1937

    todrop = []
    ## Remove citations and publications
    for year2 in range(startingyear,2023):
        if year2 not in yearstoconsider:
            if 'papers_' + str(year2) in total.columns:
                todrop.append("papers_" + str(year2))
                todrop.append("citations_" + str(year2))

    #Remove fascia_id except current year
    for year2 in range(startingyear,2023):
        if year2 != year:
            if 'fascia_id_' + str(year2) in total.columns:
                todrop.append("fascia_id_" + str(year2))

    # Sum papers and citations up until year
    startingyear = 1937
    total_papers = total['papers_' + str(startingyear)]
    total_citations = total['citations_' + str(startingyear)]
    for yyyy in range(startingyear+1,year+1):
        if 'papers_' + str(yyyy) in total.columns:
            total_papers = total_papers + total['papers_' + str(yyyy)]
            total_citations = total_citations + total['citations_' + str(yyyy)]
    

    yb = total.drop(columns=todrop)
    yb.rename(columns={'fascia_id_' + str(year) : 'fascia_id'}, inplace=True)
    yb['Total_papers'] = total_papers
    yb['Total_citations'] = total_citations

    df_yby[year] = yb

In [55]:
df_yby[2022].columns

Index(['S.S.D.', 'Name', 'anni_servizio', 'papers_2019', 'papers_2020',
       'papers_2021', 'papers_2022', 'citations_2019', 'citations_2020',
       'citations_2021', 'citations_2022', 'fascia_id', 'sex', 'Total_papers',
       'Total_citations'],
      dtype='object')

In [56]:

for y in df_yby.keys():
    df = df_yby[y]
    df.rename(columns={'papers_' + str(y-3): 'papers_-3', 'papers_' + str(y-2): 'papers_-2', 'papers_' + str(y-1): 'papers_-1', 'papers_' + str(y): 'papers_0',
    'citations_' + str(y-3): 'citations_-3', 'citations_' + str(y-2): 'citations_-2', 'citations_' + str(y-1): 'citations_-1', 'citations_' + str(y): 'citations_0'},inplace=True)


In [57]:
for y in df_yby.keys():
    one_hot = pd.get_dummies(df_yby[y]['S.S.D.'])
    df_yby[y] = df_yby[y].drop('S.S.D.',axis = 1)
    df_yby[y] = df_yby[y].join(one_hot)    


In [58]:
for year in df_yby.keys():
    df_yby[year].to_csv(os.path.join(savefolder,str(year) + ".csv"))